# 杭州萧山区项目`WRF-CMAQ`模拟分析
## 模拟结果预处理：*`MET-Data`*

---
*@author: Evan*\
*@date: 2023-08-28*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd

import sys
sys.path.append('../../src/')
from namelist import *

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [3]:
isam = xr.open_dataset('D:/data/Project_Xiaoshan/202205/COMBINE_ACONC_v54_ISAM_intel_CN3XS_135X138_2022_isam.nc')
grid = xr.open_dataset('D:/data/Project_Xiaoshan/GRIDCRO2D_2022234.nc')

In [4]:
# 将时间点转换为实际时间
times=pd.date_range(timestart,timeend,freq='h')
times

DatetimeIndex(['2022-05-01 00:00:00', '2022-05-01 01:00:00',
               '2022-05-01 02:00:00', '2022-05-01 03:00:00',
               '2022-05-01 04:00:00', '2022-05-01 05:00:00',
               '2022-05-01 06:00:00', '2022-05-01 07:00:00',
               '2022-05-01 08:00:00', '2022-05-01 09:00:00',
               ...
               '2022-05-31 14:00:00', '2022-05-31 15:00:00',
               '2022-05-31 16:00:00', '2022-05-31 17:00:00',
               '2022-05-31 18:00:00', '2022-05-31 19:00:00',
               '2022-05-31 20:00:00', '2022-05-31 21:00:00',
               '2022-05-31 22:00:00', '2022-05-31 23:00:00'],
              dtype='datetime64[ns]', length=744, freq='H')

In [5]:
# 将层数转换为气压高度
preslevel=np.array(
    [1.,     0.9979, 0.9956, 0.9931, 0.9904, 0.9875, 0.9844, 0.9807, 0.9763, 0.9711,
     0.9649, 0.9575, 0.9488, 0.9385, 0.9263, 0.912,  0.8951, 0.8753, 0.8521, 0.8251,
     0.7937, 0.7597, 0.7229, 0.6883, 0.641,  0.596,  0.5484, 0.4985, 0.4467, 0.3934,
     0.3393, 0.285,  0.2316, 0.1801, 0.1324, 0.0903, 0.0542, 0.0241,]
    )
pres=preslevel*950+50
pres

array([1000.   ,  998.005,  995.82 ,  993.445,  990.88 ,  988.125,
        985.18 ,  981.665,  977.485,  972.545,  966.655,  959.625,
        951.36 ,  941.575,  929.985,  916.4  ,  900.345,  881.535,
        859.495,  833.845,  804.015,  771.715,  736.755,  703.885,
        658.95 ,  616.2  ,  570.98 ,  523.575,  474.365,  423.73 ,
        372.335,  320.75 ,  270.02 ,  221.095,  175.78 ,  135.785,
        101.49 ,   72.895])

In [6]:
isam

<xarray.Dataset>
Dimensions:  (TSTEP: 768, VAR: 13, DATE-TIME: 2, LAY: 26, ROW: 138, COL: 135)
Dimensions without coordinates: TSTEP, VAR, DATE-TIME, LAY, ROW, COL
Data variables: (12/14)
    TFLAG    (TSTEP, VAR, DATE-TIME) int32 ...
    O3_Nor   (TSTEP, LAY, ROW, COL) float32 ...
    O3_NW    (TSTEP, LAY, ROW, COL) float32 ...
    O3_FY    (TSTEP, LAY, ROW, COL) float32 ...
    O3_SE    (TSTEP, LAY, ROW, COL) float32 ...
    O3_XS    (TSTEP, LAY, ROW, COL) float32 ...
    ...       ...
    O3_XST   (TSTEP, LAY, ROW, COL) float32 ...
    O3_XSR   (TSTEP, LAY, ROW, COL) float32 ...
    O3_XSS   (TSTEP, LAY, ROW, COL) float32 ...
    O3_OTH   (TSTEP, LAY, ROW, COL) float32 ...
    O3_ICO   (TSTEP, LAY, ROW, COL) float32 ...
    O3_BCO   (TSTEP, LAY, ROW, COL) float32 ...
Attributes: (12/33)
    IOAPI_VERSION:  ioapi-3.2: $Id: init3.F90 185 2020-08-28 16:49:45Z coats ...
    EXEC_ID:        ????????????????                                         ...
    FTYPE:          1
    CDATE:          2023240
    CTIME:          20001
    WDATE:          2023240
    ...             ...
    VGLVLS:         [1.     0.9979 0.9956 0.9931 0.9904 0.9875 0.9844 0.9807 ...
    GDNAM:          CN3XS_135X138   
    UPNAM:          COMBINE         
    VAR-LIST:       O3_Nor          O3_NW           O3_FY           O3_SE    ...
    FILEDESC:       Src Apportionment file output                            ...
    HISTORY:

In [7]:
days=1 # set spin-up days
dataset=xr.Dataset(
    data_vars=dict(
        # ! vars from ISAM
        O3_Nor=(['time','level','y','x'],isam.O3_Nor[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing','units':'ug m-3'}),
        O3_NW=(['time','level','y','x'],isam.O3_NW[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing Industry','units':'ug m-3'}),
        O3_FY=(['time','level','y','x'],isam.O3_FY[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing Trasportation','units':'ug m-3'}),
        O3_SE=(['time','level','y','x'],isam.O3_SE[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing Agricultrue','units':'ug m-3'}),
        O3_XS=(['time','level','y','x'],isam.O3_XS[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing Agricultrue','units':'ug m-3'}),
        O3_XSI=(['time','level','y','x'],isam.O3_XSI[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing Agricultrue','units':'ug m-3'}),
        O3_XSP=(['time','level','y','x'],isam.O3_XSP[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing Agricultrue','units':'ug m-3'}),
        O3_XST=(['time','level','y','x'],isam.O3_XST[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing Agricultrue','units':'ug m-3'}),
        O3_XSR=(['time','level','y','x'],isam.O3_XSR[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing Agricultrue','units':'ug m-3'}),
        O3_XSS=(['time','level','y','x'],isam.O3_XSS[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from Anqing Agricultrue','units':'ug m-3'}),
        O3_OTH=(['time','level','y','x'],isam.O3_OTH[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from OTHER','units':'ug m-3'}),
        O3_ICO=(['time','level','y','x'],isam.O3_ICO[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from ICON','units':'ug m-3'}),
        O3_BCO=(['time','level','y','x'],isam.O3_BCO[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'O3 from BCON','units':'ug m-3'}),
        ),
    coords=dict(
        time=times,
        level=pres[:26],
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Xiaoshan_202205',
        grid='CN3XS_135X138',
        createtime=pd.Timestamp.now().strftime('%Y-%m-%d'),
    ),
)
dataset

<xarray.Dataset>
Dimensions:    (time: 744, level: 26, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2022-05-01 ... 2022-05-31T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 703.9 659.0 616.2
    latitude   (y, x) float32 28.29 28.29 28.29 28.29 ... 31.89 31.89 31.89
    longitude  (y, x) float32 117.9 117.9 117.9 118.0 ... 122.3 122.3 122.4
Dimensions without coordinates: y, x
Data variables: (12/13)
    O3_Nor     (time, level, y, x) float32 8.627 9.146 9.239 ... 0.1657 0.1656
    O3_NW      (time, level, y, x) float32 10.89 11.52 11.66 ... 0.3013 0.3033
    O3_FY      (time, level, y, x) float32 5.11 5.315 5.331 ... 0.2012 0.2043
    O3_SE      (time, level, y, x) float32 4.58 3.952 3.447 ... 0.1878 0.1889
    O3_XS      (time, level, y, x) float32 4.494 4.54 4.381 ... 0.1563 0.157
    O3_XSI     (time, level, y, x) float32 3.627 3.662 3.531 ... 0.1464 0.1469
    ...         ...
    O3_XST     (time, level, y, x) float32 3.041 3.03 2.889 ... 0.1349 0.1352
    O3_XSR     (time, level, y, x) float32 0.619 0.5514 0.4969 ... 0.1243 0.1244
    O3_XSS     (time, level, y, x) float32 0.0003724 0.0003213 ... 0.1239 0.124
    O3_OTH     (time, level, y, x) float32 26.15 26.57 26.73 ... 1.556 1.572
    O3_ICO     (time, level, y, x) float32 0.6923 0.6102 ... 0.1235 0.1236
    O3_BCO     (time, level, y, x) float32 31.24 32.45 33.47 ... 128.7 127.6
Attributes:
    case:        Xiaoshan_202205
    grid:        CN3XS_135X138
    createtime:  2023-08-28

In [8]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in dataset.data_vars}
dataset.to_netcdf('D:/Download/XS_2205_isam.nc',encoding=encoding)